In [ ]:

!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.9 MB/s eta 0:00:00
time: 396 µs (started: 2024-04-28 00:04:42 +00:00)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random
from torch.nn.utils.rnn import pad_sequence


time: 642 µs (started: 2024-04-28 00:08:20 +00:00)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

time: 3.73 ms (started: 2024-04-28 00:08:22 +00:00)


In [ ]:
english_to_french = [
    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")
]

time: 1.85 ms (started: 2024-04-28 00:08:23 +00:00)


In [ ]:


# Create a French to English list by reversing the pairs in english_to_french
french_to_english = [(french, english) for english, french in english_to_french]

# Initialize a dictionary for indexing words, starting with special tokens
word_to_index = {"SOS": 0, "EOS": 1}

# Populate the dictionary with indices for each unique word in the dataset
for english_sentence, french_sentence in english_to_french:
    for word in english_sentence.split() + french_sentence.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)




time: 1.41 ms (started: 2024-04-28 00:08:24 +00:00)


In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, dataset, word_to_index):
        self.dataset = dataset
        self.word_to_index = word_to_index

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        french_sentence, english_sentence = self.dataset[idx]
        input_indices = [self.word_to_index[word] for word in french_sentence.split()] + [word_to_index["EOS"]]
        target_indices = [self.word_to_index[word] for word in english_sentence.split()] + [word_to_index["EOS"]]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

def collate_batch(batch):
    input_tensors, target_tensors = zip(*batch)
    input_tensors_padded = pad_sequence(input_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    target_tensors_padded = pad_sequence(target_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    return input_tensors_padded, target_tensors_padded

# Transformer Model
class Transformer(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers = 4, num_heads = 8, dropout = 0.2):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, src, tgt):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        output = self.transformer(src, tgt, tgt_mask=tgt_mask)
        return self.fc_out(output)


time: 1.78 ms (started: 2024-04-28 00:08:25 +00:00)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = len(word_to_index)
embed_size = 128
num_layers = 4
num_heads = 8
dropout = 0.2
epochs = 60

time: 980 µs (started: 2024-04-28 00:09:00 +00:00)


In [ ]:
model = Transformer(input_size, embed_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_function = nn.CrossEntropyLoss(ignore_index=word_to_index["EOS"])

dataset = TranslationDataset(english_to_french, word_to_index)
translation_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn= collate_batch)

for epoch_num in range(epochs + 1):
    model.train()

    total_train_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for english_tensor, french_tensor in translation_loader:
        english_tensor = english_tensor.to(device)
        french_tensor = french_tensor.to(device)

        optimizer.zero_grad()
        output = model(english_tensor, french_tensor[:, :-1])
        output_flat = output.view(-1, output.size(-1))
        french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

        valid_french_tokens = french_target_flat != 1
        loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(output_flat, 1)
        valid_predictions = (predicted == french_target_flat) & valid_french_tokens
        correct_predictions += valid_predictions.sum().item()
        total_predictions += valid_french_tokens.sum().item()

    mean_train_loss = total_train_loss / len(translation_loader)
    training_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

    model.eval()
    total_eval_loss = 0
    eval_correct = 0
    eval_total = 0
    with torch.no_grad():
        for english_tensor, french_tensor in translation_loader:
            english_tensor = english_tensor.to(device)
            french_tensor = french_tensor.to(device)

            output = model(english_tensor, french_tensor[:, :-1])
            output_flat = output.view(-1, output.size(-1))
            french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

            valid_french_tokens = french_target_flat != 1
            loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])

            total_eval_loss += loss.item()

            _, predicted = torch.max(output_flat, 1)
            valid_predictions = (predicted == french_target_flat) & valid_french_tokens
            eval_correct += valid_predictions.sum().item()
            eval_total += valid_french_tokens.sum().item()

    mean_eval_loss = total_eval_loss / len(translation_loader)
    evaluation_accuracy = eval_correct / eval_total if eval_total > 0 else 0
    if epoch_num % 10 == 0:
        print(f'Epoch {epoch_num}: Loss = {mean_train_loss:.4f}, Train Accuracy = {training_accuracy:.4f}, '
              f'Val Loss = {mean_eval_loss:.4f}, Val Accuracy = {evaluation_accuracy:.4f}')


Epoch 0: Loss = 6.0538, Train Accuracy = 0.0067, Val Loss = 5.6131, Val Accuracy = 0.0604
Epoch 10: Loss = 4.5236, Train Accuracy = 0.1544, Val Loss = 4.0826, Val Accuracy = 0.1812
Epoch 20: Loss = 3.4922, Train Accuracy = 0.2852, Val Loss = 2.8782, Val Accuracy = 0.3893
Epoch 30: Loss = 2.6271, Train Accuracy = 0.5268, Val Loss = 1.9311, Val Accuracy = 0.6611
Epoch 40: Loss = 1.9441, Train Accuracy = 0.7181, Val Loss = 1.2092, Val Accuracy = 0.8188
Epoch 50: Loss = 1.3561, Train Accuracy = 0.8893, Val Loss = 0.7275, Val Accuracy = 0.9631
Epoch 60: Loss = 0.9474, Train Accuracy = 0.9664, Val Loss = 0.4426, Val Accuracy = 0.9966
time: 6.36 s (started: 2024-04-28 00:09:01 +00:00)


In [ ]:
sentence = "You are tired"

# Ensure the model is in evaluation mode
model.eval()

# Tokenize the sentence and convert each word to its corresponding index, handling unknown words
input_indices = [word_to_index.get(word, word_to_index["EOS"]) for word in sentence.split()]
input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

# Start the output sequence with the 'SOS' token
output_indices = [word_to_index['SOS']]

# Generate output words one by one
for _ in range(len(input_indices)):
    # Prepare the current output sequence as input for the model
    output_tensor = torch.tensor(output_indices).unsqueeze(0).to(device)

    # Obtain model predictions
    with torch.no_grad():
        predictions = model(input_tensor, output_tensor)

    # Determine the predicted index (highest probability in the last position)
    predicted_index = predictions[0, -1, :].argmax(0).item()

    # Break if the model predicts the 'EOS' token
    if predicted_index == word_to_index['EOS']:
        break

    # Append the predicted index to the output sequence
    output_indices.append(predicted_index)

# Map indices back to words, skipping the 'SOS' token for the final sentence
index_to_word = {index: word for word, index in word_to_index.items()}
translated_sentence = ' '.join(index_to_word[idx] for idx in output_indices if idx != word_to_index['SOS'])

translated_sentence


'dans le fatigué'

time: 35.3 ms (started: 2024-04-28 00:09:23 +00:00)


question 4

In [ ]:
# Assume english_to_french is defined elsewhere
french_to_english = [(french, english) for english, french in english_to_french]
word_to_index = {"SOS": 0, "EOS": 1}
for eng, fre in english_to_french:
    for word in eng.split() + fre.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

time: 1.28 ms (started: 2024-04-28 00:09:26 +00:00)


In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, dataset, word_to_index):
        self.dataset = dataset
        self.word_to_index = word_to_index

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        french_sentence, english_sentence = self.dataset[idx]
        input_indices = [self.word_to_index[word] for word in french_sentence.split()] + [word_to_index["EOS"]]
        target_indices = [self.word_to_index[word] for word in english_sentence.split()] + [word_to_index["EOS"]]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

def collate_batch(batch):
    input_tensors, target_tensors = zip(*batch)
    input_tensors_padded = pad_sequence(input_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    target_tensors_padded = pad_sequence(target_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    return input_tensors_padded, target_tensors_padded

# Transformer Model
class Transformer(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers = 4, num_heads = 8, dropout = 0.2):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, src, tgt):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        output = self.transformer(src, tgt, tgt_mask=tgt_mask)
        return self.fc_out(output)

time: 1.43 ms (started: 2024-04-28 00:09:27 +00:00)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = len(word_to_index)
embed_size = 128
num_layers = 4
num_heads = 8
dropout = 0.2
epochs = 60

time: 969 µs (started: 2024-04-28 00:09:29 +00:00)


In [ ]:
model = Transformer(input_size, embed_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_function = nn.CrossEntropyLoss(ignore_index=word_to_index["EOS"])

dataset = TranslationDataset(french_to_english, word_to_index)
translation_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn= collate_batch)

for epoch_num in range(epochs + 1):
    model.train()

    total_train_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for english_tensor, french_tensor in translation_loader:
        english_tensor = english_tensor.to(device)
        french_tensor = french_tensor.to(device)

        optimizer.zero_grad()
        output = model(english_tensor, french_tensor[:, :-1])
        output_flat = output.view(-1, output.size(-1))
        french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

        valid_french_tokens = french_target_flat != 1
        loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(output_flat, 1)
        valid_predictions = (predicted == french_target_flat) & valid_french_tokens
        correct_predictions += valid_predictions.sum().item()
        total_predictions += valid_french_tokens.sum().item()

    mean_train_loss = total_train_loss / len(translation_loader)
    training_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

    model.eval()
    total_eval_loss = 0
    eval_correct = 0
    eval_total = 0
    with torch.no_grad():
        for english_tensor, french_tensor in translation_loader:
            english_tensor = english_tensor.to(device)
            french_tensor = french_tensor.to(device)

            output = model(english_tensor, french_tensor[:, :-1])
            output_flat = output.view(-1, output.size(-1))
            french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

            valid_french_tokens = french_target_flat != 1
            loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])

            total_eval_loss += loss.item()

            _, predicted = torch.max(output_flat, 1)
            valid_predictions = (predicted == french_target_flat) & valid_french_tokens
            eval_correct += valid_predictions.sum().item()
            eval_total += valid_french_tokens.sum().item()

    mean_eval_loss = total_eval_loss / len(translation_loader)
    evaluation_accuracy = eval_correct / eval_total if eval_total > 0 else 0
    if epoch_num % 10 == 0:
        print(f'Epoch {epoch_num}: Loss = {mean_train_loss:.4f}, Train Accuracy = {training_accuracy:.4f}, '
              f'Val Loss = {mean_eval_loss:.4f}, Val Accuracy = {evaluation_accuracy:.4f}')


Epoch 0: Loss = 5.9288, Train Accuracy = 0.0183, Val Loss = 5.4744, Val Accuracy = 0.0916
Epoch 10: Loss = 4.3777, Train Accuracy = 0.1868, Val Loss = 4.0205, Val Accuracy = 0.1832
Epoch 20: Loss = 3.3105, Train Accuracy = 0.3480, Val Loss = 2.7133, Val Accuracy = 0.4103
Epoch 30: Loss = 2.4379, Train Accuracy = 0.5348, Val Loss = 1.8176, Val Accuracy = 0.6300
Epoch 40: Loss = 1.7965, Train Accuracy = 0.7179, Val Loss = 1.1307, Val Accuracy = 0.8755
Epoch 50: Loss = 1.2285, Train Accuracy = 0.8791, Val Loss = 0.6646, Val Accuracy = 0.9853
Epoch 60: Loss = 0.8439, Train Accuracy = 0.9670, Val Loss = 0.3793, Val Accuracy = 1.0000
time: 7.07 s (started: 2024-04-28 00:09:30 +00:00)


In [ ]:
sentence =  "Elle est heureuse"


# Ensure the model is in evaluation mode
model.eval()

# Tokenize the sentence and convert each word to its corresponding index, handling unknown words
input_indices = [word_to_index.get(word, word_to_index["EOS"]) for word in sentence.split()]
input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

# Start the output sequence with the 'SOS' token
output_indices = [word_to_index['SOS']]

# Generate output words one by one
for _ in range(len(input_indices)):
    # Prepare the current output sequence as input for the model
    output_tensor = torch.tensor(output_indices).unsqueeze(0).to(device)

    # Obtain model predictions
    with torch.no_grad():
        predictions = model(input_tensor, output_tensor)

    # Determine the predicted index (highest probability in the last position)
    predicted_index = predictions[0, -1, :].argmax(0).item()

    # Break if the model predicts the 'EOS' token
    if predicted_index == word_to_index['EOS']:
        break

    # Append the predicted index to the output sequence
    output_indices.append(predicted_index)

# Map indices back to words, skipping the 'SOS' token for the final sentence
index_to_word = {index: word for word, index in word_to_index.items()}
translated_sentence = ' '.join(index_to_word[idx] for idx in output_indices if idx != word_to_index['SOS'])

translated_sentence


'happy is happy'

time: 28.4 ms (started: 2024-04-28 00:10:10 +00:00)


time: 75.8 ms (started: 2024-04-28 00:05:02 +00:00)
